# 如何从工具流式传输事件

:::info 前置条件

本指南假设您已熟悉以下概念：
- [LangChain 工具](/docs/concepts/tools)
- [自定义工具](/docs/how_to/custom_tools)
- [使用流事件](/docs/how_to/streaming/#using-stream-events)
- [在自定义工具中访问 RunnableConfig](/docs/how_to/tool_configure/)

:::

如果您有调用 [聊天模型](/docs/concepts/chat_models/)、[检索器](/docs/concepts/retrievers/)或其他 [runnables](/docs/concepts/runnables/) 的 [工具](/docs/concepts/tools/)，您可能希望访问这些 runnables 的内部事件或使用其他属性配置它们。本指南向您展示如何正确手动传递参数，以便使用 `astream_events()` 方法实现此目的。

:::caution 兼容性

如果您在 `python<=3.10` 中运行 `async` 代码，LangChain 无法自动将配置（包括 `astream_events()` 所需的回调）传播到子 runnable。这是您可能无法看到自定义 runnable 或工具发出事件的常见原因。

如果您运行的是 `python<=3.10`，则需要在异步环境中手动将 `RunnableConfig` 对象传播到子 runnable。有关如何手动传播配置的示例，请参阅下面 `bar` RunnableLambda 的实现。

如果您运行的是 `python>=3.11`，则 `RunnableConfig` 将在异步环境中自动传播到子 runnable。但是，如果您的代码可能在旧版 Python 中运行，手动传播 `RunnableConfig` 仍然是一个好主意。

本指南还需要 `langchain-core>=0.2.16`。
:::

假设您有一个自定义工具，它通过提示聊天模型返回仅 10 个单词的输入摘要，然后反转输出。首先，以一种简单的方式定义它：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="model" />


In [1]:
# | output: false
# | echo: false

import os
from getpass import getpass

from langchain_anthropic import ChatAnthropic

if "ANTHROPIC_API_KEY" not in os.environ:
    os.environ["ANTHROPIC_API_KEY"] = getpass()

model = ChatAnthropic(model="claude-3-5-sonnet-20240620", temperature=0)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool


@tool
async def special_summarization_tool(long_text: str) -> str:
    """使用高级技术总结输入文本的工具。"""
    prompt = ChatPromptTemplate.from_template(
        "您是一位专家作家。用 10 个单词或更少总结以下文本：\n\n{long_text}"
    )

    def reverse(x: str):
        return x[::-1]

    chain = prompt | model | StrOutputParser() | reverse
    summary = await chain.ainvoke({"long_text": long_text})
    return summary

直接调用工具可以正常工作：

In [ ]:
LONG_TEXT = """
旁白：
(黑屏显示文字；可以听到蜜蜂嗡嗡的声音)
根据所有已知的航空法则，蜜蜂不可能飞行。它的翅膀太小，无法让它胖胖的小身体离开地面。当然，蜜蜂还是飞了，因为蜜蜂不在乎人类认为不可能的事情。
BARRY BENSON：
(Barry 正在挑选一件衬衫)
黄色，黑色。黄色，黑色。黄色，黑色。黄色，黑色。哦，黑色和黄色！让我们稍微改变一下。
JANET BENSON：
Barry！早餐准备好了！
BARRY：
来了！稍等一下。
"""

await special_summarization_tool.ainvoke({"long_text": LONG_TEXT})

'.yad noitaudarg rof tiftuo sesoohc yrraB ;scisyhp seifed eeB'

但是，如果您想访问聊天模型的原始输出而不是完整工具，您可能会尝试使用 [`astream_events()`](/docs/how_to/streaming/#using-stream-events) 方法并查找 `on_chat_model_end` 事件。以下是发生的情况：

In [ ]:
stream = special_summarization_tool.astream_events({"long_text": LONG_TEXT})

async for event in stream:
    if event["event"] == "on_chat_model_end":
        # 在 python<=3.10 中永远不会触发！
        print(event)

您会注意到（除非您在 `python>=3.11` 中运行本指南），没有从子运行中发出的聊天模型事件！

这是因为上面的示例没有将工具的配置对象传递到内部链中。要解决此问题，请重新定义您的工具以接受一个特殊参数，其类型为 `RunnableConfig`（有关更多详细信息，请参阅 [本指南](/docs/how_to/tool_configure)）。您还需要在执行内部链时传递该参数：

In [ ]:
from langchain_core.runnables import RunnableConfig


@tool
async def special_summarization_tool_with_config(
    long_text: str, config: RunnableConfig
) -> str:
    """使用高级技术总结输入文本的工具。"""
    prompt = ChatPromptTemplate.from_template(
        "您是一位专家作家。用 10 个单词或更少总结以下文本：\n\n{long_text}"
    )

    def reverse(x: str):
        return x[::-1]

    chain = prompt | model | StrOutputParser() | reverse
    # 将 "config" 对象作为参数传递给任何执行的 runnables
    summary = await chain.ainvoke({"long_text": long_text}, config=config)
    return summary

现在尝试使用新工具执行与之前相同的 `astream_events()` 调用：

In [6]:
stream = special_summarization_tool_with_config.astream_events({"long_text": LONG_TEXT})

async for event in stream:
    if event["event"] == "on_chat_model_end":
        print(event)

{'event': 'on_chat_model_end', 'data': {'output': AIMessage(content='Bee defies physics; Barry chooses outfit for graduation day.', additional_kwargs={}, response_metadata={'stop_reason': 'end_turn', 'stop_sequence': None}, id='run-337ac14e-8da8-4c6d-a69f-1573f93b651e', usage_metadata={'input_tokens': 182, 'output_tokens': 19, 'total_tokens': 201, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}}), 'input': {'messages': [[HumanMessage(content="You are an expert writer. Summarize the following text in 10 words or less:\n\n\nNARRATOR:\n(Black screen with text; The sound of buzzing bees can be heard)\nAccording to all known laws of aviation, there is no way a bee should be able to fly. Its wings are too small to get its fat little body off the ground. The bee, of course, flies anyway because bees don't care what humans think is impossible.\nBARRY BENSON:\n(Barry is picking out a shirt)\nYellow, black. Yellow, black. Yellow, black. Yellow, black. Ooh, black and yellow! Let's s

太棒了！这次发出了一个事件。

对于流式处理，`astream_events()` 会在可能的情况下自动调用链中的内部 runnables 并启用流式处理，因此如果您想要从聊天模型生成的令牌流，您只需过滤以查找 `on_chat_model_stream` 事件，而无需其他更改：

In [7]:
stream = special_summarization_tool_with_config.astream_events({"long_text": LONG_TEXT})

async for event in stream:
    if event["event"] == "on_chat_model_stream":
        print(event)

{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='', additional_kwargs={}, response_metadata={}, id='run-f5e049f7-4e98-4236-87ab-8cd1ce85a2d5', usage_metadata={'input_tokens': 182, 'output_tokens': 2, 'total_tokens': 184, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})}, 'run_id': 'f5e049f7-4e98-4236-87ab-8cd1ce85a2d5', 'name': 'ChatAnthropic', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-5-sonnet-20240620', 'ls_model_type': 'chat', 'ls_temperature': 0.0, 'ls_max_tokens': 1024}, 'parent_ids': ['51858043-b301-4b76-8abb-56218e405283']}
{'event': 'on_chat_model_stream', 'data': {'chunk': AIMessageChunk(content='Bee', additional_kwargs={}, response_metadata={}, id='run-f5e049f7-4e98-4236-87ab-8cd1ce85a2d5')}, 'run_id': 'f5e049f7-4e98-4236-87ab-8cd1ce85a2d5', 'name': 'ChatAnthropic', 'tags': ['seq:step:2'], 'metadata': {'ls_provider': 'anthropic', 'ls_model_name': 'claude-3-5-sonnet-20240620', 'ls_model

## 下一步

您现在已经了解了如何从工具中流式传输事件。接下来，请查看以下指南以了解有关使用工具的更多信息：

- 传递 [运行时值到工具](/docs/how_to/tool_runtime)
- 将 [工具结果传递回模型](/docs/how_to/tool_results_pass_to_model)
- [分派自定义回调事件](/docs/how_to/callbacks_custom_events)

您还可以查看一些更具体的工具调用用途：

- 构建 [使用工具的链和代理](/docs/how_to#tools)
- 从模型获取 [结构化输出](/docs/how_to/structured_output/)